## 0. Initialization

In [6]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import os

# Initialize mediapipe pose class
mp_pose = mp.solutions.pose
pose_landmark = mp_pose.PoseLandmark
landmarks_to_draw = [landmark for landmark in mp_pose.PoseLandmark]
posture_data = {}  # Dictionary to store posture analysis results
buffer, sml_buffer = 10, 3  # Buffers for thresholding angle differences in joint analysis
sign = "| "  # Separator used in string formatting for posture conditions
choice = None

def initialize_capture():
    """
    Initializes video capture from a file or a camera based on user input.

    Returns:
    tuple: (cv2.VideoCapture object, boolean indicating if the source is an image, source path or None)
    """
    while True:
        try:
            input_choice = int(input("Enter 1 for image/video file, 2 for camera: "))
            if input_choice == 1:
                while True:  # Loop to keep asking for the file path until a valid one is provided
                    path = input("Enter the path of the image or video file: ")
                    if os.path.exists(path):
                        cap = cv2.VideoCapture(path)
                        if cap.isOpened() or path.endswith(('jpg', 'jpeg', 'png')):  # Check if it's an image or video can be opened
                            return cap, path.endswith(('jpg', 'jpeg', 'png')), path
                        else:
                            print("Failed to open the file. Please make sure it is a valid image or video file.")
                    else:
                        print("The file does not exist. Please enter a valid path.")
            elif input_choice == 2:
                return cv2.VideoCapture(int(input("Camera index (usually 0): "))), False, None
            else:
                print("Invalid choice. Please enter 1 for image/video file or 2 for camera.")
        except ValueError:
            print("Invalid input. Please enter a number (1 or 2).")

def setup_output_writer(cap, is_image):
    """
    Sets up a writer for saving the output, depending on whether the input is an image or a video.

    Args:
    cap: The VideoCapture object.
    is_image (bool): Indicates whether the input source is an image.

    Returns:
    Either a cv2.VideoWriter object for videos or a string path for images.
    """
    if is_image:
        out_path = input("Enter the path to save the processed image (include file extension): ")
        return out_path
    else:
        out_path = input("Path for output video (.avi or .mp4): ")
        frame_rate = cap.get(cv2.CAP_PROP_FPS)  # Get the frame rate of the input video
        dimensions = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))

        # Selecting the codec based on file extension
        if out_path.endswith('.mp4'):
            fourcc = cv2.VideoWriter_fourcc(*'MP4V')  # Codec for .mp4 files
        else:
            fourcc = cv2.VideoWriter_fourcc(*'XVID')  # Default codec for .avi files

        return cv2.VideoWriter(out_path, fourcc, frame_rate, dimensions)

    return None


def process_frame(frame, pose, anal_func, detect_func):
    """
    Processes a single frame for pose detection and applies analysis and detection functions.

    Args:
    frame: The current frame from the video source.
    pose: The Mediapipe pose object for detecting poses.
    anal_func (function): Function to apply pose analysis.
    detect_func (function): Function to annotate detected poses on the frame.

    Returns:
    numpy.ndarray: The processed frame with pose annotations.
    """
    # Flip the frame horizontally
    if choice == 1:
        frame = cv2.flip(frame, 1)
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image)
    if results.pose_landmarks:
        # Convert image back to BGR and apply detection function
        return detect_func(cv2.cvtColor(image, cv2.COLOR_RGB2BGR), results, *anal_func(results.pose_landmarks.landmark, mp_pose))
    return cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

def get_point(landmarks, landmark):
    """
    Extracts the x and y coordinates of a specified landmark from the landmarks.

    Args:
    landmarks (list): A list of Mediapipe pose landmarks.
    landmark (mp_pose.PoseLandmark): The specific landmark to extract.

    Returns:
    tuple: Coordinates (x, y) of the specified landmark.
    """
    return landmarks[landmark.value].x, landmarks[landmark.value].y

def calculate_angle(a, b, c, reverse=False):
    """
    Calculates the angle formed by three points, with an option to reverse the direction.

    Args:
    a, b, c (tuple): Coordinates of the points forming the angle (each as an (x, y) tuple).
    reverse (bool): If True, calculates the angle in the reverse direction.

    Returns:
    float: The calculated angle in degrees.
    """
    a, b, c = np.array(a), np.array(b), np.array(c)

    # Calculating the angle depending on the reverse flag
    if reverse:
        radians = np.arctan2(b[1] - a[1], b[0] - a[0]) - np.arctan2(c[1] - a[1], c[0] - a[0])
    else:
        radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])

    angle = np.degrees(radians)

    # Adjusting angle to the range [0, 360] and then normalizing to [-180, 180]
    angle = (angle + 360) % 360
    if angle > 180:
        angle -= 360

    return angle

def draw_colored_connection(image, results, start_idx, end_idx, color=(255, 0, 0), thickness=2):
    """
    Draws a colored line between two landmarks on the image.

    Args:
    image: The frame on which to draw the line.
    results: Mediapipe pose detection results.
    start_idx, end_idx (int): Indices of the start and end landmarks.
    color (tuple): The color of the line to draw.
    """
    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark
        start, end = landmarks[start_idx], landmarks[end_idx]
        cv2.line(image, (int(start.x * image.shape[1]), int(start.y * image.shape[0])), 
                 (int(end.x * image.shape[1]), int(end.y * image.shape[0])), color, thickness)

def calculate_posture_ratio():
    """
    Calculates the ratio of different postures detected in terms of their percentage.

    Returns:
    dict: Dictionary with the percentage of each posture condition.
    """
    section_percentages = {}
    top_level_percentages = {}

    # Calculate the total count of postures (both incorrect and correct)
    total_postures = sum(sum(postures.values()) if isinstance(postures, dict) else postures 
                         for section, postures in posture_data.items())

    # Calculate the total correct count
    correct_data = posture_data.get("Correct", {})
    correct_count = sum(correct_data.values()) if isinstance(correct_data, dict) else correct_data

    # Calculate percentages for individual conditions within each section
    for section, postures in posture_data.items():
        if section == "Correct":
            continue  # Skip the "Correct" section for this part

        section_total = sum(postures.values()) if isinstance(postures, dict) else postures
        section_percentages[section] = {condition: f"{(count / section_total) * 100:.1f}%" 
                                        for condition, count in postures.items()} if isinstance(postures, dict) else {}

    # Calculate overall issue percentages
    for section, postures in posture_data.items():
        if section == "Correct":
            continue  # Skip the "Correct" section for this part

        section_count = sum(postures.values()) if isinstance(postures, dict) else postures
        top_level_percentages[f"{section} Issue"] = f"{(section_count / total_postures) * 100:.1f}%"

    # Calculate the correct ratio
    correct_ratio = (correct_count / (total_postures+0.0000001)) * 100
    top_level_percentages["Correct Ratio"] = f"{correct_ratio:.1f}%"

    return {**section_percentages, **top_level_percentages}

def update_posture_data(section, posture_conditions, detected_postures):
    """
    Updates the posture data dictionary based on the detected postures.

    Args:
    section (str): The section/category of the posture (e.g., "Knee", "Hip").
    posture_conditions (list): List of possible conditions within the section.
    detected_postures (str): The postures detected in the current frame.
    """
    if section == "Correct":
        posture_data[section] = {"Correct": posture_data.get(section, {}).get("Correct", 0) + 1}
    else:
        detected_postures_list = [p.strip() for p in detected_postures.split(" and ") if p.strip() in posture_conditions]
        for posture in detected_postures_list:
            posture_data.setdefault(section, {}).setdefault(posture, 0)
            posture_data[section][posture] += 1
            
def draw_labeled_box(image, results, joint_landmarks, joint_names, angles, font_scale=0.3, font_thickness=1,
                     box_color=(255, 255, 255), text_color=(139, 0, 0), edge_color=(230, 216, 173)):
    """
    For each joint, calculates the position, draws a labeled text box with angle information on the image.

    Args:
    - image (numpy.ndarray): The image on which to draw.
    - results (object): The detected pose landmarks from a pose estimation model.
    - joint_landmarks (list): The list of landmark enums to draw labels for.
    - joint_names (list): The list of joint names to draw labels for.
    - angles (tuple): Tuple containing angle values for each joint.
    - font_scale, font_thickness (int): Font properties.
    - box_color, text_color, edge_color (tuple): Box, text, and edge colors.

    Returns:
    - numpy.ndarray: The image with labeled boxes drawn on it.
    """
    for joint_index, angle in enumerate(angles):
        joint_landmark = results.pose_landmarks.landmark[joint_landmarks[joint_index]]
        angle_text = f"{joint_names[joint_index]}: {round(angle, 1)}"
        text_x = int(joint_landmark.x * image.shape[1]) + 10
        text_y = int(joint_landmark.y * image.shape[0]) - 10

        # Determine the size of the text box
        text_size, _ = cv2.getTextSize(angle_text, cv2.FONT_HERSHEY_SIMPLEX, font_scale, font_thickness)
        text_width, text_height = text_size

        box_start = (text_x - 2, text_y + 2)
        box_end = (text_x + text_width + 2, text_y - text_height - 2)

        # Draw the filled rectangle (background)
        cv2.rectangle(image, box_start, box_end, box_color, cv2.FILLED)
        # Draw the border rectangle (edges)
        cv2.rectangle(image, box_start, box_end, edge_color, 1)

        # Now put the text (in specified text color)
        text_org = (text_x, text_y)
        cv2.putText(image, angle_text, text_org, cv2.FONT_HERSHEY_SIMPLEX, font_scale, text_color, font_thickness, cv2.LINE_AA)

    return image

def run_estimation(anal_func, detect_func):
    """
    Main function to run the posture estimation process.

    Args:
    anal_func (function): Function that performs the analysis of postures.
    detect_func (function): Function that detects and annotates poses on the frame.

    Returns:
    pandas.DataFrame: DataFrame containing the percentage ratios of posture conditions.
    """
    cap, is_image, input_path = initialize_capture()
    if not cap:
        return

    out = setup_output_writer(cap, is_image)

    with mp_pose.Pose(min_detection_confidence=0.95, min_tracking_confidence=0.95) as pose:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret: break

            processed_frame = process_frame(frame, pose, anal_func, detect_func)
            cv2.imshow('Mediapipe Feed', processed_frame)

            if not is_image and out:
                out.write(processed_frame)  # Save to video if out is a VideoWriter object
            elif is_image:
                cv2.imwrite(out, processed_frame)  # Save to image file if out is a string
                break  # No need to process further frames for an image

            if cv2.waitKey(1) & 0xFF == ord('q') or is_image: break

        cap.release()
        if not is_image and out:
            out.release()
        cv2.destroyAllWindows()
    
    posture_percentages = calculate_posture_ratio()
    
    
    # Flatten the nested dictionaries
    data = []
    for section, conditions in posture_percentages.items():
        if isinstance(conditions, dict):
            for condition, percentage in conditions.items():
                data.append({'Section': section, 'Condition': condition, 'Percentage': percentage})
        else:
            data.append({'Section': section, 'Condition': '', 'Percentage': conditions})
    df = pd.DataFrame(data)
    posture_data.clear()

    return df

## 1.1 Squat Front

### 1.1.1 Analysis

In [ ]:
def squat_front_analysis(landmarks, mp_pose):
    """
    Analyzes front-view squat posture based on landmarks.

    Args:
    - landmarks (list): List of detected pose landmarks.
    - mp_pose (module): Mediapipe pose module for landmark references.

    Returns:
    - tuple: Returns a tuple containing the detected posture and a tuple of angles (left knee angle, right knee angle, hip angle difference).
    """

    # Get points for hips, knees, and FOOT_INDEX
    left_hip, left_knee, left_foot_index = map(lambda lm: get_point(landmarks, lm), [pose_landmark.LEFT_HIP, pose_landmark.LEFT_KNEE, pose_landmark.LEFT_FOOT_INDEX])
    right_hip, right_knee, right_foot_index = map(lambda lm: get_point(landmarks, lm), [pose_landmark.RIGHT_HIP, pose_landmark.RIGHT_KNEE, pose_landmark.RIGHT_FOOT_INDEX])
    
    # Calculate knee angles
    left_knee_angle = calculate_angle(left_hip, left_knee, left_foot_index)
    right_knee_angle = calculate_angle(right_hip, right_knee, right_foot_index)
    
    # Calcuate hip angles
    left_hip_angle = abs(calculate_angle(right_hip, left_hip, [left_hip[0], 0]))
    right_hip_angle = abs(calculate_angle(left_hip, right_hip, [right_hip[0], 0]))
    
    # Determine if hips are level
    left_hip_level = left_hip_angle - right_hip_angle > sml_buffer
    right_hip_level = right_hip_angle - left_hip_angle > sml_buffer
    
    
    correct_degree = 140
    
    # Outward position logic
    left_knee_outward = (left_knee_angle + correct_degree) > buffer
    right_knee_outward = (right_knee_angle - correct_degree) < -buffer

    # Inward position logic
    left_knee_inward = (left_knee_angle + correct_degree) < -buffer
    right_knee_inward = (right_knee_angle - correct_degree) > buffer

    
    # update posture condition
    knee_postures = ["Knees outwards", "Knees inwards"]
    hip_postures = ["H.L. hip", "H.R. hip"]

    posture = ""
    if (left_knee_outward or right_knee_outward) and (abs(left_knee_angle) < correct_degree + buffer and abs(right_knee_angle) < correct_degree + buffer):
        posture += sign + knee_postures[0]
    elif (left_knee_inward or right_knee_inward) and (abs(left_knee_angle) < correct_degree + buffer and abs(right_knee_angle) < correct_degree + buffer):
        posture += sign + knee_postures[1]
    if left_hip_level:
        posture += sign + hip_postures[0]
    elif right_hip_level:
        posture += sign + hip_postures[1]
    if posture == "":
        posture += "| Correct"

    detected_postures = posture.split(sign)
    for detected_posture in detected_postures:
        detected_posture = detected_posture.strip()
        if detected_posture in knee_postures:
            update_posture_data("Knee", knee_postures, detected_posture)
        elif detected_posture in hip_postures:
            update_posture_data("Hip", hip_postures, detected_posture)
        elif detected_posture == "Correct":
            posture_data["Correct"] = posture_data.get("Correct", 0) + 1

    # Return posture and angles as a tuple
    return posture, (left_knee_angle, right_knee_angle, (left_hip_angle - right_hip_angle))

### 1.1.2 Detection

In [ ]:
def squat_front_detection(image, results, knee_position, angles):
    """
    Analyzes and annotates a front-view image of a squatting posture.

    Args:
    - image (numpy.ndarray): The image on which to perform the analysis and annotations.
    - results (object): The detected pose landmarks from a pose estimation model.
    - knee_position (str): Description of the knee position (e.g., "Correct", "Too forward").
    - angles (tuple): A tuple containing the angles (left knee angle, right knee angle, hip angle difference).

    Returns:
    - numpy.ndarray: The annotated image with landmarks, angles, and posture information.
    """
    # Unpack angles to get the knee angles and hip angle difference
    left_knee_angle, right_knee_angle, hip_angle_diff = angles
    if results.pose_landmarks:
        # Specify the landmarks to draw
        landmarks_to_draw = [
            mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.LEFT_KNEE, mp_pose.PoseLandmark.LEFT_FOOT_INDEX,
            mp_pose.PoseLandmark.RIGHT_HIP, mp_pose.PoseLandmark.RIGHT_KNEE, mp_pose.PoseLandmark.RIGHT_FOOT_INDEX
        ]

        # Draw the specified landmarks
        for landmark in landmarks_to_draw:
            landmark_point = results.pose_landmarks.landmark[landmark]
            cv2.circle(image, (int(landmark_point.x * image.shape[1]), int(landmark_point.y * image.shape[0])), 5, (0, 255, 0), -1)

        # Draw the lines from hip to knee to toe for both legs
        draw_colored_connection(image, results, mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.LEFT_KNEE)
        draw_colored_connection(image, results, mp_pose.PoseLandmark.LEFT_KNEE, mp_pose.PoseLandmark.LEFT_FOOT_INDEX)
        draw_colored_connection(image, results, mp_pose.PoseLandmark.RIGHT_HIP, mp_pose.PoseLandmark.RIGHT_KNEE)
        draw_colored_connection(image, results, mp_pose.PoseLandmark.RIGHT_KNEE, mp_pose.PoseLandmark.RIGHT_FOOT_INDEX)
        
        # Draw the line connecting both hips
        draw_colored_connection(image, results, mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.RIGHT_HIP)

        # Display angles near the knees
        for idx, angle in enumerate((left_knee_angle, right_knee_angle)):
            knee_landmark = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_KNEE if idx == 0 else mp_pose.PoseLandmark.RIGHT_KNEE]
            cv2.putText(image, f"{'Left' if idx == 0 else 'Right'} Knee: {abs(round(angle, 1))}", 
                        (int(knee_landmark.x * image.shape[1]), int(knee_landmark.y * image.shape[0] - 10)), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 255, 255), 1, cv2.LINE_AA)
            
        # Display the hip angle difference near the hip landmark
        hip_landmark = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP]
        cv2.putText(image, f'Hip Diff: {round(hip_angle_diff, 1)}', 
                (int(hip_landmark.x * image.shape[1]), int(hip_landmark.y * image.shape[0] - 20)), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 255, 255), 1, cv2.LINE_AA)

        # Add text for knee position if necessary
        cv2.putText(image, f'Posture: {knee_position}', (10, 30), cv2.FONT_HERSHEY_COMPLEX, 0.7, (255, 255, 255), 2, cv2.LINE_AA)
    
    return image


## 1.2 Squat Side

### 1.2.1 Analysis

In [ ]:
def squat_side_analysis(landmarks, mp_pose):
    """
    Analyzes front-view squat posture based on landmarks.

    Args:
    - landmarks (list): List of detected pose landmarks.
    - mp_pose (module): Mediapipe pose module for landmark references.

    Returns:
    - tuple: Returns a tuple containing the detected posture and a tuple of angles (left knee angle, right knee angle, hip angle difference).
    """
    # Get points for shoulders, hips, knees, and ankles
    left_shoulder, left_hip, left_knee, left_ankle = map(lambda lm: get_point(landmarks, lm), [mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_HIP, 
                                                                    mp_pose.PoseLandmark.LEFT_KNEE, mp_pose.PoseLandmark.LEFT_ANKLE])
    right_shoulder, right_hip, right_knee, right_ankle = map(lambda lm: get_point(landmarks, lm), [mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.RIGHT_HIP, 
                                                                        mp_pose.PoseLandmark.RIGHT_KNEE, mp_pose.PoseLandmark.RIGHT_ANKLE])

    
    # Analyse hip posture
    # Calculate hip angles
    left_hip_angle = abs(calculate_angle(left_shoulder, left_hip, [left_hip[0], 0], True))
    right_hip_angle = abs(calculate_angle(right_shoulder, right_hip, [right_hip[0], 0], True))

    # Analyse ankle posture
    # Calculate ankle angles
    left_ankle_angle = abs(calculate_angle(left_knee, left_ankle, [left_ankle[0], 0], True))
    right_ankle_angle = abs(calculate_angle(right_knee, right_ankle, [right_ankle[0], 0], True))
    
    # Determine if hip-shoulder and ankle-knee lines are asymmetric
    left_asymmetric = abs(left_hip_angle - left_ankle_angle) > buffer + 15
    right_asymmetric = abs(right_hip_angle - right_ankle_angle) > buffer + 15
    
    
    # Analyse knee posture
    # Calculate knee angles, subtract 180 for reflection
    left_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
    right_knee_angle = calculate_angle(right_hip, right_knee, right_ankle)
    
    # Determine if knee angles are correct
    correct_knee_angle = 38
    knee_low = any([abs(left_knee_angle) > (correct_knee_angle - buffer), abs(right_knee_angle) > (correct_knee_angle - buffer)])



    # update posture condition
    asym_postures = ["L. body asym", "R. body asym"]
    knee_postures = ["Low knees"]
    
    posture = ""
    if left_asymmetric:
        posture += sign + asym_postures[0]
    if right_asymmetric:
        posture += sign + asym_postures[1]
    if knee_low and (abs(left_knee_angle) < correct_knee_angle and abs(right_knee_angle) < correct_knee_angle):
        posture += sign + knee_postures[0]
    if posture == "":
        posture = "Correct"
        
    detected_postures = posture.split(sign)
    for detected_posture in detected_postures:
        detected_posture = detected_posture.strip()
        if detected_posture in asym_postures:
            update_posture_data("Symmetry", asym_postures, detected_posture)
        elif detected_posture in knee_postures:
            update_posture_data("Knee", knee_postures, detected_posture)
        elif detected_posture == "Correct":
            posture_data["Correct"] = posture_data.get("Correct", 0) + 1

        
    return posture, (left_knee_angle, right_knee_angle), (left_hip_angle, right_hip_angle, left_ankle_angle, right_ankle_angle)


### 1.2.2 Detection

In [ ]:
def squat_side_detection(image, results, posture, angles, body_tilts):
    """
    Analyzes and annotates a side-view image of a squatting posture.

    Args:
    - image (numpy.ndarray): The image on which to perform the analysis and annotations.
    - results (object): The detected pose landmarks from a pose estimation model.
    - posture (str): Description of the overall posture.
    - angles (list): A list of knee angles.
    - body_tilts (list): List of body tilts (left and right hip and ankle angles).

    Returns:
    - numpy.ndarray: The annotated image with landmarks and angles.
    """
    if results.pose_landmarks:
        # Specify the landmarks to draw
        landmarks_to_draw = [
            mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_ELBOW, mp_pose.PoseLandmark.LEFT_WRIST, mp_pose.PoseLandmark.LEFT_HIP,  
            mp_pose.PoseLandmark.LEFT_KNEE, mp_pose.PoseLandmark.LEFT_ANKLE, mp_pose.PoseLandmark.LEFT_FOOT_INDEX,
            mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.RIGHT_ELBOW, mp_pose.PoseLandmark.RIGHT_WRIST, mp_pose.PoseLandmark.RIGHT_HIP, 
            mp_pose.PoseLandmark.RIGHT_KNEE, mp_pose.PoseLandmark.RIGHT_ANKLE,
        ]

        # Draw the lines with specified colors
        draw_colored_connection(image, results, landmarks_to_draw[0], landmarks_to_draw[3], color=(0, 0, 255))
        
        draw_colored_connection(image, results, landmarks_to_draw[3], landmarks_to_draw[4])
        draw_colored_connection(image, results, mp_pose.PoseLandmark.LEFT_WRIST, mp_pose.PoseLandmark.LEFT_SHOULDER)
        draw_colored_connection(image, results, mp_pose.PoseLandmark.LEFT_KNEE, mp_pose.PoseLandmark.LEFT_ANKLE, color=(0, 0, 255))
        draw_colored_connection(image, results, mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.RIGHT_HIP, color=(0, 0, 255))
        draw_colored_connection(image, results, mp_pose.PoseLandmark.RIGHT_HIP, mp_pose.PoseLandmark.RIGHT_KNEE)
        draw_colored_connection(image, results, mp_pose.PoseLandmark.RIGHT_KNEE, mp_pose.PoseLandmark.RIGHT_ANKLE, color=(0, 0, 255))
        draw_colored_connection(image, results, mp_pose.PoseLandmark.RIGHT_WRIST, mp_pose.PoseLandmark.RIGHT_SHOULDER)
        
        

        # Display angles for knees, hips, and ankles
        for side_index, angle in enumerate(angles):
            knee_landmark = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_KNEE if side_index == 0 else mp_pose.PoseLandmark.RIGHT_KNEE]
            cv2.putText(image, f"{'Left' if side_index == 0 else 'Right'} Knee: {abs(round(angle, 1))}", 
                        (int(knee_landmark.x * image.shape[1]), int(knee_landmark.y * image.shape[0] - 10)), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 255, 255), 1, cv2.LINE_AA)

        # Display body tilts
        tilt_labels = ['LUBT', 'RUBT', 'LLBT', 'RLBT']
        for index, tilt in enumerate(body_tilts):
            tilt_point = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP if index < 2 else mp_pose.PoseLandmark.LEFT_ANKLE]
            cv2.putText(image, f"{tilt_labels[index]}: {round(tilt, 1)}", 
                        (int(tilt_point.x * image.shape[1]), int(tilt_point.y * image.shape[0] + 20 * (index % 2))), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 255, 255), 1, cv2.LINE_AA)

        # Add text for knee position if necessary
        cv2.putText(image, f'Posture: {posture}', (0, 20), cv2.FONT_HERSHEY_COMPLEX, 0.7, (255, 255, 255), 2, cv2.LINE_AA)

    return image


In [ ]:
def squat_back_detection(image, results, posture, angles):
    if results.pose_landmarks:
        # Specify the landmarks to draw
        landmarks_to_draw = [
            mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_HIP, 
            mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.RIGHT_HIP
        ]
        
        # Draw lines between shoulders and hips
        draw_colored_connection(image, results, landmarks_to_draw[0], landmarks_to_draw[2])
        draw_colored_connection(image, results, landmarks_to_draw[1], landmarks_to_draw[3])
        draw_colored_connection(image, results, landmarks_to_draw[0], landmarks_to_draw[1], color=(0, 0, 255))
        draw_colored_connection(image, results, landmarks_to_draw[2], landmarks_to_draw[3], color=(0, 0, 255))

        
        # Draw the specified landmarks
        for landmark in landmarks_to_draw:
            landmark_point = results.pose_landmarks.landmark[landmark]
            pos = (int(landmark_point.x * image.shape[1]), int(landmark_point.y * image.shape[0]))
            cv2.circle(image, pos, 3, (255, 255, 255), -1)

        # Draw the lines and display angles
        joint_names = ["Shoulder Dev", "Hip Dev"]
        joint_landmarks = [landmark.value for landmark in landmarks_to_draw[:2]]


        # Call the draw_labeled_box function
        image = draw_labeled_box(image, results, joint_landmarks, joint_names, angles)

        # Add text for posture
        cv2.putText(image, f'Posture: {posture}', (20, 50), cv2.FONT_HERSHEY_COMPLEX, 0.7, (255, 255, 255), 2)

    return image


In [ ]:
# Squat side
run_estimation(squat_side_analysis, squat_side_detection)

## 1.3 Squat Back 

### 1.3.1 Analysis

In [ ]:
def squat_back_analysis(landmarks, mp_pose):
    """
    Analyzes back-view squat posture based on landmarks.

    Args:
    - landmarks (list): List of detected pose landmarks.
    - mp_pose (module): Mediapipe pose module for landmark references.

    Returns:
    - tuple: Returns a tuple containing the detected posture and a tuple of angles (shoulder angle difference, hip angle difference).
    """

    # Get points for shoulders and hips
    left_shoulder, left_hip = map(lambda lm: get_point(landmarks, lm), [pose_landmark.LEFT_SHOULDER, pose_landmark.LEFT_HIP])
    right_shoulder, right_hip = map(lambda lm: get_point(landmarks, lm), [pose_landmark.RIGHT_SHOULDER, pose_landmark.RIGHT_HIP])
    
    # Posture Analysis
    # Calculate shoulder angles
    left_shoulder_angle = abs(calculate_angle(right_shoulder, left_shoulder, [left_shoulder[0], 0]))
    right_shoulder_angle = abs(calculate_angle(left_shoulder, right_shoulder, [right_shoulder[0], 0]))
    
    # Determine if shoulders are level
    left_shoulder_high = left_shoulder_angle - right_shoulder_angle > sml_buffer
    right_shoulder_high = right_shoulder_angle - left_shoulder_angle > sml_buffer

     # Calculate hip angles
    left_hip_angle = abs(calculate_angle(right_hip, left_hip, [left_hip[0], 0]))
    right_hip_angle = abs(calculate_angle(left_hip, right_hip, [right_hip[0], 0]))
    
    # Determine if hips are level
    left_hip_high = left_hip_angle - right_hip_angle > sml_buffer
    right_hip_high = right_hip_angle - left_hip_angle > sml_buffer
    

    # Update posture condition
    shoulder_postures = ["H.L. shoulder", "H.R. shoulder"]
    hip_postures = ["H.L. hip", "H.R. hip"]
    posture = ""
    if left_shoulder_high:
        posture += sign + shoulder_postures[0]
    elif right_shoulder_high:
        posture += sign + shoulder_postures[1]
    if left_hip_high:
        posture += sign + hip_postures[0]
    elif right_hip_high:
        posture += sign + hip_postures[1]
    if posture == "":
        posture = "Correct"

    detected_postures = posture.split(sign)
    for detected_posture in detected_postures:
        detected_posture = detected_posture.strip()
        if detected_posture in shoulder_postures:
            update_posture_data("Shoulder", shoulder_postures, detected_posture)
        elif detected_posture in hip_postures:
            update_posture_data("Hip", hip_postures, detected_posture)
        elif detected_posture == "Correct":
            posture_data["Correct"] = posture_data.get("Correct", 0) + 1     
        
        
    return posture, (left_shoulder_angle - right_shoulder_angle, left_hip_angle - right_hip_angle)

### 1.3.2 Detection

In [ ]:
def squat_back_detection(image, results, posture, angles):
    if results.pose_landmarks:
        # Specify the landmarks to draw
        landmarks_to_draw = [
            mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_HIP, 
            mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.RIGHT_HIP
        ]
        
        # Draw lines between shoulders and hips
        draw_colored_connection(image, results, landmarks_to_draw[0], landmarks_to_draw[2])
        draw_colored_connection(image, results, landmarks_to_draw[1], landmarks_to_draw[3])
        draw_colored_connection(image, results, landmarks_to_draw[0], landmarks_to_draw[1], color=(0, 0, 255))
        draw_colored_connection(image, results, landmarks_to_draw[2], landmarks_to_draw[3], color=(0, 0, 255))

        
        # Draw the specified landmarks
        for landmark in landmarks_to_draw:
            landmark_point = results.pose_landmarks.landmark[landmark]
            pos = (int(landmark_point.x * image.shape[1]), int(landmark_point.y * image.shape[0]))
            cv2.circle(image, pos, 3, (255, 255, 255), -1)

        # Draw the lines and display angles
        joint_names = ["Shoulder Dev", "Hip Dev"]
        joint_landmarks = [landmark.value for landmark in landmarks_to_draw[:2]]


        # Call the draw_labeled_box function
        image = draw_labeled_box(image, results, joint_landmarks, joint_names, angles)

        # Add text for posture
        cv2.putText(image, f'Posture: {posture}', (20, 50), cv2.FONT_HERSHEY_COMPLEX, 0.7, (255, 255, 255), 2)

    return image


## 2. Knee Raising

### 2.0.1 Analysis

In [53]:
def knee_raising_analysis(landmarks, mp_pose):
    """
    Analyzes posture during knee-raising exercises based on landmarks.

    Args:
    - landmarks (list): List of detected pose landmarks.
    - mp_pose (module): Mediapipe pose module for landmark references.

    Returns:
    - tuple: Returns a tuple containing the detected posture and a list with shoulder angle difference, shoulder midpoint x-coordinate, and central vertical line x-coordinate.
    """
    # Get points for shoulders and hips
    left_shoulder, left_hip = map(lambda lm: get_point(landmarks, lm), [pose_landmark.LEFT_SHOULDER, pose_landmark.LEFT_HIP])
    right_shoulder, right_hip = map(lambda lm: get_point(landmarks, lm), [pose_landmark.RIGHT_SHOULDER, pose_landmark.RIGHT_HIP])
    
    
    
    # Posture Analysis
    # Calculate shoulder angles
    left_shoulder_angle = abs(calculate_angle(right_shoulder, left_shoulder, [left_shoulder[0], 0]))
    right_shoulder_angle = abs(calculate_angle(left_shoulder, right_shoulder, [right_shoulder[0], 0]))
    
    # Determine if shoulders are level
    left_shoulder_high = left_shoulder_angle - right_shoulder_angle > sml_buffer
    right_shoulder_high = right_shoulder_angle - left_shoulder_angle > sml_buffer

    
    
    # Calculate the midpoint of the shoulders
    shoulder_mid_x = (left_shoulder[0] + right_shoulder[0]) / 2
    # Calculate the central vertical line (midpoint between hips)
    central_vertical_line_x = (left_hip[0] + right_hip[0]) / 2
    
    mp_buffer = 0.025 # set buffer for midpoint
    # Determine if the shoulder mid point is over the left or right of the hip mid point
    mid_point_left = (shoulder_mid_x - central_vertical_line_x) < -mp_buffer
    mid_point_right = (shoulder_mid_x - central_vertical_line_x) > mp_buffer
    #print((shoulder_mid_x - central_vertical_line_x))
    # Update posture condition
    shoulder_postures = ["H.L. shoulder", "H.R. shoulder"]
    mp_postures = ["Body too left", "Body too right"]
    
    posture = ""
    if left_shoulder_high:
        posture += sign + shoulder_postures[0]
    elif right_shoulder_high:
        posture += sign + shoulder_postures[1]
    if mid_point_left:
        posture += sign + mp_postures[0]
    elif mid_point_right:
        posture += sign + mp_postures[1]
    if posture == "":
        posture = "Correct"

    detected_postures = posture.split(sign)
    for detected_posture in detected_postures:
        detected_posture = detected_posture.strip()
        if detected_posture in shoulder_postures:
            update_posture_data("Shoulder", shoulder_postures, detected_posture)
        elif detected_posture in mp_postures:
            update_posture_data("Mid Point", mp_postures, detected_posture)
        elif detected_posture == "Correct":
            posture_data["Correct"] = posture_data.get("Correct", 0) + 1 

    return posture, [left_shoulder_angle - right_shoulder_angle, shoulder_mid_x, central_vertical_line_x]

### 2.0.2 Detection

In [62]:
def knee_raising_detection(image, results, posture, analysis_results):
    """
    Analyzes and annotates an image for knee raising exercises.

    Args:
    - image (numpy.ndarray): The image on which to perform the analysis and annotations.
    - results (object): The detected pose landmarks from a pose estimation model.
    - posture (str): Description of the overall posture.
    - analysis_results (tuple): Contains analysis data like shoulder vertical difference,
                                shoulder midpoint, and central vertical line position.

    Returns:
    - numpy.ndarray: The annotated image with landmarks, vertical line, and deviation information.
    """
    shoulder_vertical_difference, shoulder_mid_x, central_vertical_line_x = analysis_results

    if results.pose_landmarks:
        landmark_idx = [11, 12, 23, 24]
        
        # Draw the lines
        draw_colored_connection(image, results, landmarks_to_draw[landmark_idx[0]], landmarks_to_draw[landmark_idx[1]])
        draw_colored_connection(image, results, landmarks_to_draw[landmark_idx[2]], landmarks_to_draw[landmark_idx[3]], color=(0, 0, 255))
        draw_colored_connection(image, results, landmarks_to_draw[landmark_idx[0]], landmarks_to_draw[landmark_idx[2]], color=(0, 0, 255))
        draw_colored_connection(image, results, landmarks_to_draw[landmark_idx[1]], landmarks_to_draw[landmark_idx[3]], color=(0, 0, 255))

        
        # Draw central vertical line
        central_vertical_line_pixel_x = int(central_vertical_line_x * image.shape[1])
        cv2.line(image, (central_vertical_line_pixel_x, 0), (central_vertical_line_pixel_x, image.shape[0]), (255, 255, 0), 1)

        # Calculate middle dot position (midpoint between shoulders)
        middle_dot_x = int(shoulder_mid_x * image.shape[1])
        middle_dot_y = int((results.pose_landmarks.landmark[landmarks_to_draw[landmark_idx[0]]].y +
                            results.pose_landmarks.landmark[landmarks_to_draw[landmark_idx[1]]].y) / 2 * image.shape[0])
        
        # Draw the specified landmarks
        for idx in landmark_idx:
            landmark_point = results.pose_landmarks.landmark[landmarks_to_draw[idx]]
            pos = (int(landmark_point.x * image.shape[1]), int(landmark_point.y * image.shape[0]))
            cv2.circle(image, pos, 3, (255, 255, 255), -1)
            
        # Draw the middle dot
        cv2.circle(image, (middle_dot_x, middle_dot_y), 3, (0, 255, 0), -1)  # Green dot
        
        # Calculate positions for drawing text
        shoulder_midpoint_pos = (middle_dot_x, middle_dot_y)
    
        # For "Shoulder Diff" text
        shoulder_diff_text = f'Diff: {round(abs(shoulder_vertical_difference), 1)}'
        deviation_text = 'Left' if (shoulder_mid_x - central_vertical_line_x) > 0 else 'Right' if (shoulder_mid_x - central_vertical_line_x) < 0 else 'Centered'
        deviation_full_text = f'Dev: {deviation_text}'

        # Calculate size of the text for background box calculation
        shoulder_diff_text_size = cv2.getTextSize(shoulder_diff_text, cv2.FONT_HERSHEY_SIMPLEX, 0.3, 1)[0]
        deviation_text_size = cv2.getTextSize(deviation_full_text, cv2.FONT_HERSHEY_SIMPLEX, 0.3, 1)[0]

        # Shoulder Diff Box
        shoulder_diff_box_start = (shoulder_midpoint_pos[0] + 5, shoulder_midpoint_pos[1] - 5 - shoulder_diff_text_size[1] - 2)
        shoulder_diff_box_end = (shoulder_diff_box_start[0] + shoulder_diff_text_size[0] + 4, shoulder_midpoint_pos[1] - 5 + 2)
        cv2.rectangle(image, shoulder_diff_box_start, shoulder_diff_box_end, (255, 255, 255), cv2.FILLED)
        cv2.rectangle(image, shoulder_diff_box_start, shoulder_diff_box_end, (230, 216, 173), 1)

        # Deviation Box
        deviation_box_start = (central_vertical_line_pixel_x + 5, shoulder_midpoint_pos[1] + 15 - deviation_text_size[1] - 2)
        deviation_box_end = (deviation_box_start[0] + deviation_text_size[0] + 4, shoulder_midpoint_pos[1] + 15 + 2)
        cv2.rectangle(image, deviation_box_start, deviation_box_end, (255, 255, 255), cv2.FILLED)
        cv2.rectangle(image, deviation_box_start, deviation_box_end, (230, 216, 173), 1)

        # Now put the text on top of the boxes
        cv2.putText(image, shoulder_diff_text, (shoulder_diff_box_start[0], shoulder_diff_box_start[1] + shoulder_diff_text_size[1] + 2), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (139, 0, 0), 1)
        cv2.putText(image, deviation_full_text, (deviation_box_start[0], deviation_box_start[1] + deviation_text_size[1] + 2), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (139, 0, 0), 1)
        
        
        # Add text for posture
        cv2.putText(image, f'Posture: {posture}', (10, 30), cv2.FONT_HERSHEY_COMPLEX, 0.7, (255, 255, 255), 2)

    return image


In [63]:
# Knee raising
run_estimation(knee_raising_analysis, knee_raising_detection)

Enter 1 for image/video file, 2 for camera: 1
Enter the path of the image or video file: 1.mp4
Path for output video (.avi or .mp4): 


,Section,Condition,Percentage
0,Shoulder,H.L. shoulder,100.0%
1,Mid Point,Body too right,100.0%
2,Shoulder Issue,,52.0%
3,Mid Point Issue,,7.3%
4,Correct Ratio,,40.7%


## 3.1 Stand front

### 3.1.1 Analysis

In [10]:
def stand_front_analysis(landmarks, mp_pose):
    """
    Analyzes front-view standing posture based on landmarks.

    Args:
    - landmarks (list): List of detected pose landmarks.
    - mp_pose (module): Mediapipe pose module for landmark references.

    Returns:
    - tuple: Returns a tuple containing the detected posture and a tuple of angle differences for ears, shoulders, hips, knees, and ankles.
    """
    # Get points for ears, shoulders, hips, knees, and ankles
    left_ear, left_shoulder, left_hip, left_knee, left_ankle = map(
        lambda lm: get_point(landmarks, lm),
        [mp_pose.PoseLandmark.LEFT_EAR, mp_pose.PoseLandmark.LEFT_SHOULDER, 
         mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.LEFT_KNEE, 
         mp_pose.PoseLandmark.LEFT_ANKLE])

    right_ear, right_shoulder, right_hip, right_knee, right_ankle = map(
        lambda lm: get_point(landmarks, lm),
        [mp_pose.PoseLandmark.RIGHT_EAR, mp_pose.PoseLandmark.RIGHT_SHOULDER, 
         mp_pose.PoseLandmark.RIGHT_HIP, mp_pose.PoseLandmark.RIGHT_KNEE, 
         mp_pose.PoseLandmark.RIGHT_ANKLE])

    # Calculate joint angles
    left_ear_angle = abs(calculate_angle([left_ear[0], 0], left_ear, right_ear))
    right_ear_angle = abs(calculate_angle([right_ear[0], 0], right_ear, left_ear))

    left_shoulder_angle = abs(calculate_angle([left_shoulder[0], 0], left_shoulder, right_shoulder))
    right_shoulder_angle = abs(calculate_angle([right_shoulder[0], 0], right_shoulder, left_shoulder))

    left_hip_angle = abs(calculate_angle([left_hip[0], 0], left_hip, right_hip))
    right_hip_angle = abs(calculate_angle([right_hip[0], 0], right_hip, left_hip))

    left_knee_angle = abs(calculate_angle([left_knee[0], 0], left_knee, right_knee))
    right_knee_angle = abs(calculate_angle([right_knee[0], 0], right_knee, left_knee))

    left_ankle_angle = abs(calculate_angle([left_ankle[0], 0], left_ankle, right_ankle))
    right_ankle_angle = abs(calculate_angle([right_ankle[0], 0], right_ankle, left_ankle))

    # Determine if joints are level using angle differences and buffer

    ear_postures = ["H.L. ear", "H.R. ear"]
    shoulder_postures = ["H.L. shoulder", "H.R. shoulder"]
    hip_postures = ["H.L. hip", "H.R. hip"]
    knee_postures = ["H.L. knee", "H.R. knee"]
    ankle_postures = ["H.L. ankle", "H.R. ankle"]

    posture = ""
    if left_ear_angle - right_ear_angle > sml_buffer:
        posture += sign + ear_postures[0]
    elif right_ear_angle - left_ear_angle > sml_buffer:
        posture += sign + ear_postures[1]

    if left_shoulder_angle - right_shoulder_angle > sml_buffer:
        posture += sign + shoulder_postures[0]
    elif right_shoulder_angle - left_shoulder_angle > sml_buffer:
        posture += sign + shoulder_postures[1]

    if left_hip_angle - right_hip_angle > sml_buffer:
        posture += sign + hip_postures[0]
    elif right_hip_angle - left_hip_angle > sml_buffer:
        posture += sign + hip_postures[1]

    if left_knee_angle - right_knee_angle > sml_buffer:
        posture += sign + knee_postures[0]
    elif right_knee_angle - left_knee_angle > sml_buffer:
        posture += sign + knee_postures[1]

    if left_ankle_angle - right_ankle_angle > sml_buffer:
        posture += sign + ankle_postures[0]
    elif right_ankle_angle - left_ankle_angle > sml_buffer:
        posture += sign + ankle_postures[1]

    if posture == "":
        posture = "Correct"

    # Update posture data
    detected_postures = posture.split(sign)
    for detected_posture in detected_postures:
        detected_posture = detected_posture.strip()
        if detected_posture in ear_postures:
            update_posture_data("Ear", ear_postures, detected_posture)
        elif detected_posture in shoulder_postures:
            update_posture_data("Shoulder", shoulder_postures, detected_posture)
        elif detected_posture in hip_postures:
            update_posture_data("Hip", hip_postures, detected_posture)
        elif detected_posture in knee_postures:
            update_posture_data("Knee", knee_postures, detected_posture)
        elif detected_posture in ankle_postures:
            update_posture_data("Ankle", ankle_postures, detected_posture)
        elif detected_posture == "Correct":
            posture_data["Correct"] = posture_data.get("Correct", 0) + 1

    return posture, (left_ear_angle - right_ear_angle, left_shoulder_angle - right_shoulder_angle, left_hip_angle - right_hip_angle, left_knee_angle - right_knee_angle, left_ankle_angle - right_ankle_angle)

# Define or import calculate_angle, posture_data, update_posture_data, mp_pose, and pose_landmark before using this function.


### 3.1.2 Detection

In [49]:
def stand_front_detection(image, results, posture, angles):
    """
    Analyzes and annotates an image for stand front exercises.

    Args:
    - image (numpy.ndarray): The image on which to perform the analysis and annotations.
    - results (object): The detected pose landmarks from a pose estimation model.
    - posture (str): Description of the overall posture.
    - analysis_results (tuple): Contains analysis data like shoulder vertical difference,
                                shoulder midpoint, and central vertical line position.

    Returns:
    - numpy.ndarray: The annotated image with landmarks, vertical line, and deviation information.
    """
    
    if results.pose_landmarks:
        landmark_idx = [7, 8, 11, 12, 23, 24, 25, 26, 27, 28]
        # Draw the lines
        for i in range(0, len(landmark_idx), 2):
            draw_colored_connection(image, results, landmarks_to_draw[landmark_idx[i]], landmarks_to_draw[landmark_idx[i+1]])
        for i in range(2, len(landmark_idx)-2, 2):
            draw_colored_connection(image, results, landmarks_to_draw[landmark_idx[i]], landmarks_to_draw[landmark_idx[i+2]], color=(0, 0, 255))
        for i in range(3, len(landmark_idx)-2, 2):
            draw_colored_connection(image, results, landmarks_to_draw[landmark_idx[i]], landmarks_to_draw[landmark_idx[i+2]], color=(0, 0, 255))

        # Draw the specified landmarks
        for idx in landmark_idx:
            landmark_point = results.pose_landmarks.landmark[landmarks_to_draw[idx]]
            pos = (int(landmark_point.x * image.shape[1]), int(landmark_point.y * image.shape[0]))
            cv2.circle(image, pos, 3, (255, 255, 255), -1)

        # Draw the lines and display angles
        joint_names = ["Ear", "Shoulder", "Hip", "Knee", "Ankle"]
        joint_landmarks = [landmarks_to_draw[idx].value for idx in landmark_idx[::2]]

        # Call the draw_labeled_box function
        image = draw_labeled_box(image, results, joint_landmarks, joint_names, angles)
        
        # Add text for posture
        cv2.putText(image, f'Posture: {posture}', (0, 20), cv2.FONT_HERSHEY_COMPLEX, 0.7, (255, 255, 255), 2, cv2.LINE_AA)

    return image

## 3.2 Stand Side

### Analysis

In [7]:
def stand_side_analysis(landmarks, mp_pose):
    """
    Analyzes side-view standing posture based on landmarks.

    Args:
    - landmarks (list): List of detected pose landmarks.
    - mp_pose (module): Mediapipe pose module for landmark references.

    Returns:
    - tuple: Returns a tuple containing the detected posture and the left shoulder angle.
    """

    # Get points for ear and shoulder
    left_ear, left_shoulder, left_hip, left_knee, left_ankle = map(
        lambda lm: get_point(landmarks, lm),
        [mp_pose.PoseLandmark.LEFT_EAR, mp_pose.PoseLandmark.LEFT_SHOULDER, 
         mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.LEFT_KNEE, mp_pose.PoseLandmark.LEFT_ANKLE])

    # Posture Analysis
    # Calculate shoulder angle
    left_shoulder_angle = abs(calculate_angle([left_shoulder[0], 0], left_shoulder, left_ear))
    
    left_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)

    # Determine if shoulder angle is correct
    correct_shoulder_angle = 0  # Correct shoulder angle (ideal posture)
    shoulder_high = left_shoulder_angle > (correct_shoulder_angle + buffer)

    # Update posture condition
    shoulder_postures = ["Forward head"]
    posture = ""
    if shoulder_high:
        posture += sign + shoulder_postures[0]


    if posture == "":
        posture = "Correct"

    # Update posture data
    detected_posture = posture.strip(sign)
    if detected_posture in shoulder_postures:
        update_posture_data("Head", shoulder_postures, detected_posture)
    elif detected_posture == "Correct":
        posture_data["Correct"] = posture_data.get("Correct", 0) + 1

    return posture, (left_shoulder_angle, left_knee_angle)

### 3.2.2 Detection

In [64]:
def stand_side_detection(image, results, posture, angles):
    """
    Analyzes and annotates a side-view image of a standing posture.

    Args:
    - image (numpy.ndarray): The image on which to perform the analysis and annotations.
    - results (object): The detected pose landmarks from a pose estimation model.
    - posture (str): Description of the overall posture.
    - angles (tuple): Tuple containing angles, where angles[0] is the neck angle and angles[1] is another angle, such as the knee angle.

    Returns:
    - numpy.ndarray: The annotated image with neck angle and posture information.
    """
    if results.pose_landmarks:
        landmark_idx = [7, 11, 23, 25, 27]
        # Draw the lines
        draw_colored_connection(image, results, landmarks_to_draw[7], landmarks_to_draw[11])
        draw_colored_connection(image, results, landmarks_to_draw[11], landmarks_to_draw[23], color=(0, 0, 255))
        draw_colored_connection(image, results, landmarks_to_draw[23], landmarks_to_draw[25])
        draw_colored_connection(image, results, landmarks_to_draw[25], landmarks_to_draw[27], color=(0, 0, 255))
        
        # Draw a vertical line across the left hip
        left_hip_landmark = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP.value]
        left_hip_x = int(left_hip_landmark.x * image.shape[1])
        cv2.line(image, (left_hip_x, 0), (left_hip_x, image.shape[0]), (255, 255, 0), thickness=1)
        
        # Draw circles for specified landmarks
        
        for idx in landmark_idx:
            landmark_point = results.pose_landmarks.landmark[landmarks_to_draw[idx]]
            pos = (int(landmark_point.x * image.shape[1]), int(landmark_point.y * image.shape[0]))
            cv2.circle(image, pos, 3, (255, 255, 255), -1)
            
        # Display angle information using a loop
        joint_names = ['Neck', 'Knee']  # Add more joint names as needed
        joint_landmarks = [landmarks_to_draw[11].value, landmarks_to_draw[25].value]

        # Call the draw_labeled_box function
        image = draw_labeled_box(image, results, joint_landmarks, joint_names, angles)

        # Add text for posture
        cv2.putText(image, f'Posture: {posture}', (10, 20), cv2.FONT_HERSHEY_COMPLEX, 0.7, (255, 255, 255), 2)

    return image


# Run Main Function

In [ ]:
# Squat front
run_estimation(squat_front_analysis, squat_front_detection)

In [ ]:
# Squat side
run_estimation(squat_side_analysis, squat_side_detection)

In [ ]:
# Squat back
run_estimation(squat_back_analysis, squat_back_detection)

In [ ]:
# Knee raising
run_estimation(knee_raising_analysis, knee_raising_detection)

In [ ]:
# Stand front
run_estimation(stand_front_analysis, stand_front_detection)

In [65]:
# Stand side
run_estimation(stand_side_analysis, stand_side_detection)

Enter 1 for image/video file, 2 for camera: 1
Enter the path of the image or video file: 4.mp4
Path for output video (.avi or .mp4): 


,Section,Condition,Percentage
0,Head,Forward head,100.0%
1,Head Issue,,100.0%
2,Correct Ratio,,0.0%
